# Loading the `Moral-Stories` dataset
***
The dataset and code can be found <a href="https://github.com/demelin/moral_stories">here</a>.\
The authors provide 12k unique norms and, for some reason, additional 700k variations of the same norms, just with NaN fields every now and then. Zero additional information, but maybe I am overlooking something here?
* Might be for different tasks? But then they only provide a single label which is always 1 for any NaN rows...

# Sample task: Action classification
***
For starters, let's reproduce a task from the paper:
* Given an action, predict whether it is moral or immoral.
* For simplicity, we do not use the splits introduced in the paper, but rather random splitting

We start by loading the data as a `pandas.DataFrame`:

In [ ]:
from ailignment.datasets.moral_stories import get_moral_stories, make_action_classification_dataframe
from ailignment.datasets import get_accuracy_metric, join_sentences, tokenize_and_split
from transformers import TrainingArguments
import pandas as pd
import datasets
import transformers
from ailignment import sequence_classification

transformers.logging.set_verbosity_warning()

dataframe = get_moral_stories()

In [ ]:
results = []
for wd in [0,0.01,0.05,0.1]:
    for lr in [1e-5, 5e-5]:
        test_split = 0.2
        batch_size = 12
        model = "distilbert-base-uncased"
        #model = "albert-base-v2"
        action_dataframe = make_action_classification_dataframe(dataframe)
        input_columns = ["action"]
        action_dataframe["task_input"] = join_sentences(action_dataframe, input_columns, " ")
        dataset = datasets.Dataset.from_pandas(action_dataframe)
        dataset = dataset.train_test_split(test_size=test_split)

        def data_all(tokenizer):
            return tokenize_and_split(dataset, tokenizer, "task_input")
        def data_small(tokenizer):
            train, test = data_all(tokenizer)
            train = train.shuffle(seed=42).select(range(1000))
            test = test.shuffle(seed=42).select(range(1000))
            return train, test

        training_args = TrainingArguments(
            output_dir="results/",
            num_train_epochs=3,
            per_device_train_batch_size=batch_size,
            per_device_eval_batch_size=batch_size,
            warmup_steps=500,
            weight_decay=wd,
            learning_rate=lr,
            logging_dir='./logs',
            logging_steps=500,
            save_steps=1000000,
            save_total_limit=0,
            evaluation_strategy="epoch",
        )

        r = sequence_classification(data_all, model, training_args, get_accuracy_metric())
        acc = [x["eval_accuracy"] for x in r if "eval_accuracy" in x]
        results.append(r)
        print(wd, lr, acc)

In [ ]:
[[x["eval_accuracy"] for x in r if "eval_accuracy" in x] for r in results]

# WIP: Get score output from LM
***
Question: Is there a better way to sample from generated LM outputs?

In [ ]:
from transformers import (
    AutoModelForSequenceClassification, DistilBertTokenizerFast,
     Trainer, TrainingArguments, AutoModelWithLMHead, AutoTokenizer,
)
import torch

model = "distilbert-base-uncased"
model = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model)
model = AutoModelWithLMHead.from_pretrained(model)

prompt = "Today the weather is really nice and I am planning on "
inputs = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")

prompt_length = len(tokenizer.decode(inputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=True))
outputs = model.generate(inputs, max_length=250, do_sample=False, top_p=0.95, top_k=60,
                        return_dict_in_generate=True, output_attentions=False,
                        output_hidden_states=True, output_scores=True)
#generated = prompt + tokenizer.decode(outputs[0])[prompt_length:]

p = torch.softmax(outputs.scores[0], dim=1)

print(p.max())

# WIP: Data augmentation with NER
***
Idea: Use Named entity recognition to find and replace persons etc. 

In [ ]:
from ailignment.datasets.moral_stories import get_moral_stories, make_action_classification_dataframe
from ailignment import join_sentences, tokenize_and_split, get_accuracy_metric
dataframe = get_moral_stories()
columns = dataframe.columns[1:]
print("Running NER on columns", columns.to_list())

In [ ]:
texts = join_sentences(dataframe ,columns, "\n")

In [ ]:
import spacy
from spacy import displacy
from tqdm import tqdm
import pandas as pd

nlp = spacy.load("en_core_web_sm")

ner_pipe = nlp.pipe(tqdm(texts), disable=['tagger', 'parser', 'attribute_ruler', 'lemmatizer'])
docs = [x for x in ner_pipe]

displacy.render(docs[0], style="ent")

In [ ]:
from collections import Counter

def get_frequent_entity(doc, entity="PERSON", n=1):
    '''
    Returns the highest number of occurences of an
    entity in the NER doc.
    '''
    occurences = [(x.text, x.label_) for x in doc.ents if x.label_ == entity]
    c = Counter(occurences)
    ents = []
    for item, count in c.most_common(n):
        ents.append([x for x in doc.ents if (x.text, x.label_) == item])
    
    if n == 1 and len(ents) != 0:
        ents = ents[0]
    return ents

In [ ]:
persons = [get_frequent_entity(x, "PERSON",n=1) for x in docs]
# we are interested in the simplest case, where the NER found
# exactly 6 matches
matches = [x for x in persons if len(x) == 6]
print(f"Found {len(matches)} matches")

In [ ]:
m = matches[0]
displacy.render(m[0].doc, "ent")

In [ ]:
def replace_entity(ents, s):
    '''
    Replaces all occurences of entities in `ents` with `s`.
    `ents` is a list of entities as returned by `doc.ents`
    from an NER pipeline, they need to be from the same doc!
    '''
    offset = 0
    text = ents[0].doc.text
    new_text = ""
    for ent in ents:
        start = ent.start_char
        end = ent.end_char
        left = text[offset:start]
        new_text += left + s
        offset = end
    new_text += text[offset:]
    return new_text

In [ ]:
n_docs = [replace_entity(m, "Niklas").split("\n") for m in matches]

In [ ]:
n_docs = [m[0].doc.text.split("\n") for m in matches]

In [ ]:
dataframe_replaced = pd.DataFrame(n_docs)
dataframe_replaced.columns = columns
dataframe_replaced.head()

In [ ]:
import datasets
test_split = 0.2
batch_size = 8

action_dataframe = make_action_classification_dataframe(dataframe_replaced)

input_columns = ["action"]
action_dataframe["task_input"] = join_sentences(action_dataframe, input_columns, " ")
dataset = datasets.Dataset.from_pandas(action_dataframe)
dataset = dataset.train_test_split(test_size=test_split)


In [ ]:

from transformers import (
    AutoModelForSequenceClassification, DistilBertTokenizerFast,
     Trainer, TrainingArguments, AutoModelWithLMHead, AutoTokenizer,
)
import torch

model = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model)
model = AutoModelForSequenceClassification.from_pretrained(model)

train_data, test_data = tokenize_and_split(dataset, tokenizer, "task_input")

# for prototyping, optional
small_train_data = train_data.shuffle(seed=42).select(range(1000))
small_test_data = test_data.shuffle(seed=42).select(range(1000))

training_args = TrainingArguments(
    output_dir="results/",
    num_train_epochs=5,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=8,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=50,                # how often to log
    save_steps=1000,
    save_total_limit=0,
    evaluation_strategy="epoch",     # when to run evaluation
)

In [ ]:
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=small_train_data,   # training dataset
    eval_dataset=small_test_data,     # evaluation dataset
    compute_metrics=get_accuracy_metric,     # code to run accuracy metric
)
trainer.train()

In [ ]:
from gender_guesser.detector import Detector

In [ ]:
d = Detector()

In [ ]:
d.get_gender("Jamie")

In [3]:
from ailignment.datasets.moral_stories import _lemmatize, get_moral_stories

In [15]:
import spacy
from string import punctuation
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import pandas as pd

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kiehne\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'textcat'])
STOP_WORDS = stopwords.words('english')
stories = get_moral_stories()
columns = ["moral_action", "immoral_action"]


In [6]:
series = stories[columns[0]]

In [7]:
def lemmatize_series(series, nlp, STOP_WORDS=None):
    '''
    Given a series of strings, returns a DataFrame(["lemmas", "tokens", "maps"])
    of the lemmatized strings according to `_lemmatize` function.
    '''
    # get rid of whitespace
    translation_table = str.maketrans(' ', ' ', punctuation)
    series = series.map(lambda x: x.translate(translation_table))
    series = series.map(lambda x: _lemmatize(x, nlp, STOP_WORDS))
    data = pd.DataFrame(series.to_list(), columns=["lemmas", "tokens", "maps"])
    return data

In [8]:
y = series.map(lambda x: _lemmatize(x, nlp, STOP_WORDS))

In [24]:
data = pd.DataFrame(y.to_list(), columns=["lemmas", "tokens", "maps"])


In [25]:
data

,lemmas,tokens,maps
0,"[kevin, decide, <STPWRD>, place, <STPWRD>, wag...","[Kevin, decides, to, place, a, wager, once, ev...","{0: [0], 1: [1], 2: [2], 3: [3], 4: [4], 5: [5..."
1,"[curtis, explain, <STPWRD>, <STPWRD>, woman, <...","[Curtis, explains, to, the, woman, that, he's,...","{0: [0], 1: [1], 2: [2], 3: [3], 4: [4], 5: [5..."
2,"[anna, decide, <STPWRD>, attend, <STPWRD>, wed...","[Anna, decides, to, attend, the, wedding, even...","{0: [0], 1: [1], 2: [2], 3: [3], 4: [4], 5: [5..."
3,"[greg, tell, james, <STPWRD>, <STPWRD>, <STPWR...","[Greg, tells, James, that, he, can, play, tomo...","{0: [0], 1: [1], 2: [2], 3: [3], 4: [4], 5: [5..."
4,"[ben, call, <STPWRD>, tow, truck, <STPWRD>, ge...","[Ben, calls, a, tow, truck, and, gets, the, ca...","{0: [0], 1: [1], 2: [2], 3: [3], 4: [4], 5: [5..."
...,...,...,...
11994,"[payton, ask, <STPWRD>, friend, <STPWRD>, <STP...","[Payton, asks, her, friend, where, she, bought...","{0: [0], 1: [1], 2: [2], 3: [3], 4: [4], 5: [5..."
11995,"[helen, <STPWRD>, <STPWRD>, conversation, <STP...","[Helen, has, a, conversation, with, her, husba...","{0: [0], 1: [1], 2: [2], 3: [3], 4: [4], 5: [5..."
11996,"[cole, move, away, <STPWRD>, <STPWRD>, vehicle...","[Cole, moves, away, from, the, vehicle, that, ...","{0: [0], 1: [1], 2: [2], 3: [3], 4: [4], 5: [5..."
11997,"[harry, ask, steve, <STPWRD>, <STPWRD>, <STPWR...","[Harry, asks, Steve, what, he, should, be, doi...","{0: [0], 1: [1], 2: [2], 3: [3], 4: [4], 5: [5..."


In [18]:
b

1

In [19]:
c

2